In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Wczytanie danych Fashion-MNIST
(images, labels), (images_test, labels_test) = tf.keras.datasets.fashion_mnist.load_data()
images = images / 255.0  # Normalizacja pikseli

# Rozszerzenie wymiaru danych wejściowych, aby były kompatybilne z Conv2D
images = images.reshape(-1, 28, 28, 1)

# Podział danych na treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=10, stratify=labels)

def build_advanced_model():
    # Budowa zaawansowanego modelu sieci konwolucyjnej
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        layers.Dropout(0.5),  # Większy dropout dla lepszej generalizacji
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Budowa i trenowanie modelu
model = build_advanced_model()

# Augmentacja danych
augmenter = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
augmented_train_data = augmenter.flow(X_train, y_train, batch_size=64)

# Callbacki: EarlyStopping i ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

# Trenowanie modelu
history = model.fit(augmented_train_data, epochs=50, validation_data=(X_test, y_test),
                    callbacks=[early_stopping, reduce_lr], verbose=1)

# Ewaluacja modelu
accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
print(f"Test accuracy: {accuracy:.4f}")

# Zapisanie modelu do pliku
model.save("fashion_mnist_model.h5")

# Wypisanie architektury modelu
model.summary()

# Interfejs do predykcji i wizualizacji
def predict_and_visualize(model, image, label):
    plt.imshow(image.squeeze(), cmap='gray')
    plt.title(f"Actual: {label}")
    plt.show()

    image = np.expand_dims(image, axis=0)  # Dodanie wymiaru batch
    prediction = np.argmax(model.predict(image), axis=-1)[0]
    print(f"Predicted: {prediction}")

# Test interfejsu
sample_image = X_test[0]
sample_label = y_test[0]
model = tf.keras.models.load_model("fashion_mnist_model.h5")
predict_and_visualize(model, sample_image, sample_label)


Epoch 1/50
844/844 [==============================] - 62s 72ms/step - loss: 1.0161 - accuracy: 0.7444 - val_loss: 0.7446 - val_accuracy: 0.7802 - lr: 0.0010
Epoch 2/50
844/844 [==============================] - 63s 75ms/step - loss: 0.6505 - accuracy: 0.8175 - val_loss: 0.6619 - val_accuracy: 0.8007 - lr: 0.0010
Epoch 3/50
844/844 [==============================] - 66s 78ms/step - loss: 0.5405 - accuracy: 0.8410 - val_loss: 0.4856 - val_accuracy: 0.8582 - lr: 0.0010
Epoch 4/50
844/844 [==============================] - 65s 77ms/step - loss: 0.5034 - accuracy: 0.8502 - val_loss: 0.4394 - val_accuracy: 0.8737 - lr: 0.0010
Epoch 5/50
844/844 [==============================] - 65s 77ms/step - loss: 0.4777 - accuracy: 0.8590 - val_loss: 0.4257 - val_accuracy: 0.8673 - lr: 0.0010
Epoch 6/50
844/844 [==============================] - 63s 75ms/step - loss: 0.4675 - accuracy: 0.8633 - val_loss: 0.5771 - val_accuracy: 0.8453 - lr: 0.0010
Epoch 7/50
844/844 [==============================] - 65s 